In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
import keras

Using TensorFlow backend.


In [2]:
#Data corresponds to an emotion tag, a 48x48 greyscale picture flattened, and a tag indicating if the image belongs to the test set or the training set.
df = pd.read_csv('./data/fer2013.csv')

train = df[df['Usage']=='Training']
test = df[df['Usage']=='PublicTest']

#TODO: We might want to augment data with ImageDataGenerator 

#TODO: We might want to do some image pre-processing


#Splits data in training and testing, as well as formatting data to right input and output shapes

X_train = np.array(train.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48,1).astype('float32')))
X_test = np.array(test.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48,1).astype('float32')))

#Turn the array of 3D arrays into a 4D array
X_train = np.stack(X_train, axis=0)
X_test = np.stack(X_test, axis=0)

y_train = to_categorical(train['emotion'])
y_test =  to_categorical(test['emotion'])

#Splits off a validation set
X_val = X_train[-4000:]
y_val = y_train[-4000:]
X_train = X_train[:-4000]
y_train = y_train[:-4000]

print(y_train)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]]


In [3]:
#Define dimensions of input and output layers
D_in = X_train[0].shape
D_out = tuple(y_train[0].shape)
print(D_in)
print(D_out[0])

#Sets up model. To Bobby: Can we make this smarter so you and I can do different models?
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(tuple(D_in))))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(D_out[0], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#TODO: Train networks, evaluate results.

(48, 48, 1)
7


In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 46, 46, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 21, 21, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)         

In [5]:
print('# Fit model on training data')
history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs=50,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(X_val, y_val)
                   )

print('\nhistory dict:', history.history)


# Fit model on training data
Train on 24709 samples, validate on 4000 samples
Epoch 1/50
24709/24709 [==============================] - 21s 842us/step - loss: 2.0294 - accuracy: 0.2572 - val_loss: 1.7089 - val_accuracy: 0.3315
Epoch 2/50
24709/24709 [==============================] - 20s 829us/step - loss: 1.6499 - accuracy: 0.3587 - val_loss: 1.5749 - val_accuracy: 0.3995
Epoch 3/50
24709/24709 [==============================] - 21s 861us/step - loss: 1.5512 - accuracy: 0.4055 - val_loss: 1.5512 - val_accuracy: 0.3923
Epoch 4/50
24709/24709 [==============================] - 21s 855us/step - loss: 1.5000 - accuracy: 0.4290 - val_loss: 1.5158 - val_accuracy: 0.4275
Epoch 5/50
24709/24709 [==============================] - 21s 857us/step - loss: 1.4336 - accuracy: 0.4547 - val_loss: 1.4381 - val_accuracy: 0.4552
Epoch 6/50
24709/24709 [==============================] - 21s 835us/step - loss: 1.3853 - accuracy: 0.4755 - val_loss: 1.3707 - val_accuracy: 0.4762
Epoch 7/50
24709/24709 [====

KeyboardInterrupt: 